### This is the code to access IPEA data to get Brazilian revenue and Gross Fixed Capital Formation info

#### Data sources:
- Brazil. (2024a). Ipeadata [dataset]. http://www.ipeadata.gov.br/Default.aspx
Note: To get monthly "Indicador IPEA de formação bruta de capital fixo (FBCF): índice real encadeado (média1995 = 100)" manually access http://www.ipeadata.gov.br/Default.aspx because the API is not providing the monthly data.


#### Libraries:
- Borelli, L. (2023). Package ipeadatapy (0.1.9) [Python]. https://pypi.org/project/ipeadatapy/#description - Borelli, L. (2024). Ipeadatapy’s documentation. https://www.luanborelli.net/ipeadatapy/docs/index.html

#### Explanation of the data:

Revenue from federal taxes - gross revenue
Frequency: Monthly from 1993.01 to 2023.12
Source: Ministry of Economy, Federal Revenue Secretariat (Min. Economy/SRF)
Unit: R$ (millions)
Comment: Sum of revenues managed by the Federal Revenue Service with revenues managed by other agencies. More information: https://www.gov.br/receitafederal/pt-br/acesso-a-informacao/dados-abertos/receitadata/arrecadacao/a>.


IPEA Gross Fixed Capital Formation (GFCF) Index: Chained Real Index (average 1995 = 100)
Frequency: Monthly from 1996.01 to 2023.12
Source: Institute of Applied Economic Research
Unit: -
Comment: Gross Fixed Capital Formation (GFCF) of the economy consists of investments in machinery and equipment and in construction. Since investments in machinery and equipment are equivalent to Apparent Consumption (AC) of capital goods, a monthly indicator of GFCF was constructed by means of the weighted average of this AC indicator with a construction indicator. Base is the value, at a given moment (effective or resulting from the average taken within a time interval) that serves as a term of comparison when calculating a succession of index numbers. Here the base was fixed as the average of observations in the year 1995. Note: Updates may result in changes to previous data. More information: Indicators of Apparent Consumption of Industrial Goods: methodology and results and Rodrigues, Brazilian Statistical Dictionary, 2nd edition, IBGE Foundation, 1970.

In [14]:
# Import libraries
import ipeadatapy
import useful_functions as uf

In [15]:
# Get the list of series
series_list = ipeadatapy.list_series()
# Filter the list of series to find the desired ones
filtered_list_revenue = series_list[series_list['NAME'].str.contains('Arrecadação das receitas federais - receita bruta', case=False)]
filtered_list_FBCF = series_list[series_list['NAME'].str.contains('formação bruta de capital fixo - índice real encadeado \(média 1995 = 100\)', case=False)]

# Find the code for the desired list
desired_code_revenue = filtered_list_revenue['CODE'].values[0]
desired_code_FBCF = filtered_list_FBCF['CODE'].values[0]

# Initial date
initial_year = '2001'

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
C:\Users\augus\AppData\Local\Temp\ipykernel_1928\1306487369.py:5: SyntaxWarning: invalid escape sequence '\('
  filtered_list_FBCF = series_list[series_list['NAME'].str.contains('formação bruta de capital fixo - índice real encadeado \(média 1995 = 100\)', case=False)]


In [16]:
# Get the data
df_revenue_raw = ipeadatapy.timeseries(series = desired_code_revenue)
df_FBCF_raw = ipeadatapy.timeseries(series = desired_code_FBCF)

In [17]:
# Copy the dataframe
df_revenue = df_revenue_raw.copy()
df_FBCF = df_FBCF_raw.copy()

# Filter the data to start from the initial year
df_revenue = df_revenue[df_revenue.index >= initial_year]
df_FBCF = df_FBCF[df_FBCF.index >= initial_year]
# Format index to month and year
df_revenue.index = df_revenue.index.strftime('%m-%Y')
df_FBCF.index = df_FBCF.index.strftime('%m-%Y')

# Filter only the desired column
df_revenue = df_revenue[['VALUE (R$)']].astype(float)
df_FBCF = df_FBCF[['VALUE (-)']].astype(float)

# Rename the column
df_revenue.rename(columns={'VALUE (R$)': 'ECO_revenue'}, inplace=True)
df_FBCF.rename(columns={'VALUE (-)': 'ECO_FBCF'}, inplace=True)
# rename index 
df_revenue.index.names = ['Date']
df_FBCF.index.names = ['Date']


In [18]:
# Convert the dataframe to monthly
df_revenue = uf.convert_df_to_monthly(df_revenue, '%m-%Y')
df_FBCF = uf.convert_df_to_monthly(df_FBCF, '%m-%Y')

# Combine the dataframes
#df_revenue_FBCF = pd.concat([df_revenue, df_FBCF], axis=1)

'''
Save the dataframe to a csv file - Since the FBGC series is not providing monthly data, we will colect it manually
the reference for this data is http://www.ipeadata.gov.br/Default.aspx
It's necessary to manually download the data and save it in the data folder


#df_revenue_FBCF.to_csv('../data/df_revenue_FBCF.csv')
'''
#Save the dataframe to a csv file
df_revenue.to_csv('../data/df_revenue_FBCF.csv')